In [1]:
rm -rf obitools4
git clone https://github.com/metabarcoding/obitools4 

Cloning into 'obitools4'...
remote: Enumerating objects: 7574, done.
remote: Counting objects: 100% (591/591), done.
remote: Compressing objects: 100% (397/397), done.
remote: Total 7574 (delta 287), reused 384 (delta 148), pack-reused 6983 (from 2)
Receiving objects: 100% (7574/7574), 188.68 MiB | 17.94 MiB/s, done.
Resolving deltas: 100% (4721/4721), done.
Updating files: 100% (529/529), done.


In [2]:
### TEST FIXER LA VERSION 
cd obitools4
git reset --hard 996ec69
cd ..

Updating files: 100% (186/186), done.
HEAD is now at 996ec69 update the release notes for version 4.4.0


In [3]:
conda install -c conda-forge compilers=1.10 make=4.4.1 go=1.24.5 zlib=1.3.1 jq=1.8.1 --yes

Retrieving notices: done
Channels:
 - conda-forge
 - bioconda
Platform: linux-64
Solving environment: done


==> WARNING: A newer version of conda exists. <==
    current version: 25.3.1
    latest version: 25.5.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - compilers=1.10
    - go=1.24.5
    - jq=1.8.1
    - make=4.4.1
    - zlib=1.3.1


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _go_select-2.2.0           |            nocgo           5 KB  conda-forge
    binutils-2.43              |       h4852527_5          34 KB  conda-forge
    binutils_linux-64-2.43     |       h4852527_4          35 KB  conda-forge
    c-compiler-1.10.0          |       h2b85faf_0           7 KB  conda-forge
    ca-certificates-2025.7.14  |       hbd8a1cb_0         152 KB  conda-forge

In [4]:
# CHANGER LE LIEN DU GITLAB EN GITHUB
find . -type f -exec sed -i \
  -e 's|git\.metabarcoding\.org/obitools/obitools4/obitools4|github.com/metabarcoding/obitools4|g' \
  -e 's|git\.metabarcoding\.org/org-asm/org-asm\.git|github.com/metabarcoding/org-asm.git|g' \
  -e 's|git\.metabarcoding\.org/lecasofts/go/obitools|github.com/metabarcoding/obitools4|g' \
  -e 's|https://git\.metabarcoding\.org|https://github.com/metabarcoding|g' \
  {} +


In [5]:
export CC=$CONDA_PREFIX/bin/x86_64-conda-linux-gnu-cc
export CGO_ENABLED=1
export CGO_CFLAGS="-I$CONDA_PREFIX/include"
export CGO_LDFLAGS="-L$CONDA_PREFIX/lib"

In [6]:
cd obitools4
make
cd ..

mkdir -p build
Update version.go : pkg/obioptions/version.go to Release 4.4.0 (996ec69)
- Building obitool obiannotate...Done.
- Building obitool obiclean...Done.
- Building obitool obicleandb...Done.
- Building obitool obicomplement...Done.
- Building obitool obiconsensus...Done.
- Building obitool obiconvert...Done.
- Building obitool obicount...Done.
- Building obitool obicsv...Done.
- Building obitool obidemerge...Done.
- Building obitool obidistribute...Done.
- Building obitool obigrep...Done.
- Building obitool obijoin...Done.
- Building obitool obikmermatch...Done.
- Building obitool obikmersimcount...Done.
- Building obitool obilandmark...Done.
- Building obitool obimatrix...Done.
- Building obitool obimicrosat...Done.
- Building obitool obimultiplex...Done.
- Building obitool obipairing...Done.
- Building obitool obipcr...Done.
- Building obitool obireffamidx...Done.
- Building obitool obirefidx...Done.
- Building obitool obiscript...Done.
- Building obitool obisplit...Done.
-

In [ ]:
## FORMATAGE DES DONNES / ESPACES RETIRE
shopt -s nullglob
for f in galaxy_inputs/*/*.{fastq*,fasta}; do
  [ -e "$f" ] || continue
  dir=$(dirname -- "$f")
  base=$(basename -- "$f")
  # on enlève les espaces
  newbase=${base// /_}
  newpath="$dir/$newbase"
  [ "$newpath" = "$f" ] && continue
  echo "Renomme: $f -> $newpath"
  mv -- "$f" "$newpath"
done

In [ ]:
jq '.forward[0].path |= gsub(" "; "_")' galaxy_inputs/galaxy_inputs.json >  galaxy_inputs/galaxy_inputs_forward.json
jq '.reverse[0].path |= gsub(" "; "_")' galaxy_inputs/galaxy_inputs.json >  galaxy_inputs/galaxy_inputs_reverse.json

# Utilise les indices [0] pour accéder aux premiers éléments des tableaux

forward_path=$(jq -r '.forward[0].path' "galaxy_inputs/galaxy_inputs_forward.json")

reverse_path=$(jq -r '.reverse[0].path' "galaxy_inputs/galaxy_inputs_reverse.json")

echo "Forward path: $forward_path"
echo "Reverse path: $reverse_path"

# Traitement du nom de fichier
forward=$(basename -- "$forward_path")
filename="${forward%.*}"
format="${forward##*.}"

reverse=$(basename -- "$reverse_path")
filename="${reverse%.*}"


echo "Nom de fichier (sans chemin, sans extension) : $filename"
ouput_file="outputs/paired".$format
echo "l'ouput est" $ouput_file

Forward path: galaxy_inputs/forward/wolf_F.fastq.fastq
Reverse path: galaxy_inputs/reverse/wolf_R.fastq.fastq
Nom de fichier (sans chemin, sans extension) : wolf_F.fastq
Extension détectée : fastq
l'ouput est outputs/paired.fastq


In [ ]:
if [[ -n "$forward_path"  && "$forward_path" != "null" ]]; then
    forward_path="-F $forward_path"
    echo "$forward_path"
fi

if [[ -n "$reverse_path"  && "$reverse_path" != "null" ]]; then
    reverse_path="-R $reverse_path"
    echo "$reverse_path"
fi

minident=$(jq -r '.minident' galaxy_inputs/galaxy_inputs.json)
if [[ -n "$minident"  && "$minident" != "null" ]]; then
    minident="--min-identity=$minident"
    echo "$minident"
fi

#min overlap INTEGER 
overlap=$(jq -r '.overlap' galaxy_inputs/galaxy_inputs.json)
if [[ -n "$overlap" && "$overlap" != "null" ]]; then 
    overlap="--min-overlap=$overlap"
    echo "$overlap"
fi

-F galaxy_inputs/forward/wolf_F.fastq.fastq
-R galaxy_inputs/reverse/wolf_R.fastq.fastq


In [ ]:
./obitools4/build/obipairing $overlap $minident $forward_path $reverse_path > $ouput_file

INFO[0000] Number of workers set 120                    
INFO[0000] Found 1 files to process                     
INFO[0000] /data/jwd06/pulsar_staging/86493175/inputs/dataset_bf44bafe-d3ad-4694-8860-65b0ecd89afe.dat mime type: text/fastq 
INFO[0000] Found 1 files to process                     
INFO[0000] /data/jwd06/pulsar_staging/86493175/inputs/dataset_233a6d23-dcd8-46c9-8118-b3f3b3d3262f.dat mime type: text/fastq 
INFO[0000] Start of the sequence Pairing using 120 workers 
INFO[0000] On output use JSON headers                   
INFO[0000] Output is done on stdout                     
INFO[0000] Data is writen to stdout                     
/ Reading sequences (3882/-, 22385 it/s) [0s] INFO[0000] Initializing the DNA Scoring matrix          
- Reading sequences (42697/-, 11024 it/s) [3s] 
- Reading sequences (41393/-, 10229 it/s) [4s] 
INFO[0007] End of the sequence Pairing                  
